# Introdução

# Dependências

In [43]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install requests

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
import pandas as pd
import numpy as np
import requests
import tweepy

import json
import os

# Coleta

## Previsões

In [45]:
predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
filename = 'image_predictions.tsv'
if not os.path.exists(filename):
    response = requests.get(predictions_url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [47]:
df_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
df_predictions_clean = df_predictions.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Tweets

In [48]:
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_clean = df_tweets.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Contagem de retweets e favoritos

In [49]:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_CONSUMER_SECRET']
access_token = os.environ['TWITTER_ACCESS_TOKEN']
access_token_secret = os.environ['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitter_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [50]:
def get_retweet_fav_counts(row):
    tweet_id = row['tweet_id']
    try:
        tweet_status = twitter_api.get_status(row['tweet_id'], tweet_mode='extended')
        return (tweet_id, tweet_status.retweet_count, tweet_status.favorite_count)
    except tweepy.TweepError:
        return (tweet_id, None, None)

In [38]:
filename = 'tweet_counts.csv'
if not os.path.exists(filename):
    tweet_counts = df_tweets_clean.apply(lambda row: get_retweet_fav_counts(row), axis=1)
    df_tweet_counts = pd.DataFrame.from_records(list(tweet_counts), columns=['tweet_id', 'retweet_count', 'favorite_count'])
    df_tweet_counts.to_csv(filename, index=False)
else:
    df_tweet_counts = pd.read_csv(filename)

In [59]:
df_tweet_counts_clean = df_tweet_counts.copy()

# Avaliação

## Contagem de retweets e favoritos

In [61]:
df_tweet_counts_clean[(df_tweet_counts_clean.retweet_count.isnull()) | (df_tweet_counts_clean.favorite_count.isnull())]

,tweet_id,retweet_count,favorite_count
19,888202515573088257,NaN,NaN
48,882992080364220416,NaN,NaN
95,873697596434513921,NaN,NaN
118,869988702071779329,NaN,NaN
132,866816280283807744,NaN,NaN
155,861769973181624320,NaN,NaN
247,845459076796616705,NaN,NaN
260,842892208864923648,NaN,NaN
298,837012587749474308,NaN,NaN
382,827228250799742977,NaN,NaN


## Resposta a outros tweets

In [69]:
df_tweets_clean[df_tweets_clean.in_reply_to_status_id.notnull() | df_tweets_clean.in_reply_to_user_id.notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


# Limpeza

## Contagem de retweets e favoritos

In [62]:
df_tweet_counts_clean['retweet_count'] = df_tweet_counts_clean['retweet_count'].fillna(0).astype(int)
df_tweet_counts_clean['favorite_count'] = df_tweet_counts_clean['favorite_count'].fillna(0).astype(int)
df_tweet_counts_clean[(df_tweet_counts_clean.retweet_count.isnull()) | (df_tweet_counts_clean.favorite_count.isnull())]

,tweet_id,retweet_count,favorite_count
